In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import json
import pprint as pp

import torch
import torch.optim as optim
from tensorboard_logger import Logger as TbLogger

from critic_network import CriticNetwork
from options import get_options
from test_function import validate, get_inner_model
from baselines import CriticBaseline
from attention_model import AttentionModel
from utils import torch_load_cpu, load_model, maybe_cuda_model, load_problem
import math

## Test instance with changed graph size, seed, steps

In [2]:
%run options --graph_size 50 --seed 1234 --steps 1000 --eval_only --load_path outputs/tsp_50/run/epoch-198.pt

In [3]:
# Set the random seed
torch.manual_seed(opts.seed)

# Figure out what's the problem
problem = load_problem(opts.problem)

val_dataset = problem.make_dataset(size=opts.graph_size, num_samples=opts.val_size, filename=opts.val_dataset)
# torch.save(val_dataset,'multi.pt')
# val_dataset=torch.load('multi.pt')[0:1000]
# val_dataset=val_dataset[0:1000]
# val_dataset = val_dataset[0:1000]

In [4]:
# Load data from load_path
load_data = {}
assert opts.load_path is None or opts.resume is None, "Only one of load path and resume can be given"
load_path = opts.load_path if opts.load_path is not None else opts.resume
if load_path is not None:
    print('  [*] Loading data from {}'.format(load_path))
    load_data = load_data = torch_load_cpu(load_path)

# Initialize model
model_class = AttentionModel
model = maybe_cuda_model(
    model_class(
        opts.embedding_dim,
        opts.hidden_dim,
        problem,
        n_encode_layers=opts.n_encode_layers,
        mask_inner=True,
        mask_logits=True,
        normalization=opts.normalization,
        tanh_clipping=opts.tanh_clipping
    ),
    opts.use_cuda
)

# Overwrite model parameters by parameters to load
model_ = get_inner_model(model)
model_.load_state_dict({**model_.state_dict(), **load_data.get('model', {})})

# Initialize baseline
baseline = CriticBaseline(
            maybe_cuda_model(
                CriticNetwork(
                    2,
                    opts.embedding_dim,
                    opts.hidden_dim,
                    opts.n_encode_layers,
                    opts.normalization
                ),
                opts.use_cuda
            )
        )
# Load baseline from data, make sure script is called with same type of baseline
if 'baseline' in load_data:
    baseline.load_state_dict(load_data['baseline'])
l=[]
if opts.eval_only:
    for i in range(8):
        best = validate(model, val_dataset, opts)
        l.append(best)


    
    print('Best solutions: {} +- {}'.format(
    best.mean().item(), torch.std(best) / math.sqrt(len(best))))

  [*] Loading data from outputs/tsp_50/run/epoch-197.pt
Validating...


/home/yaoxin/.local/lib/python3.5/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Test, took 00:03:03 s
Validating...
Test, took 00:03:02 s
Validating...
Test, took 00:02:49 s
Validating...
Test, took 00:02:49 s
Validating...
Test, took 00:02:56 s
Validating...
Test, took 00:04:26 s
Validating...
Test, took 00:04:31 s
Validating...
Test, took 00:04:29 s
Best solutions: 5.745500087738037 +- 0.008316120132803917
